In [6]:
from sklearn.datasets import load_iris
#save data information as variable
iris = load_iris()

In [14]:
import pandas as pd
#make sure to save the data frame to a variable
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Add the target variable to the DataFrame
iris_data['target'] = iris.target
iris_data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [16]:
from sklearn.model_selection import train_test_split
train,test= train_test_split(iris_data, test_size=0.2, random_state=42)

In [17]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

In [2]:
!pip install sagemaker

                                              0.0/1.5 MB ? eta -:--:--
     ---                                      0.1/1.5 MB 4.2 MB/s eta 0:00:01
     ------------                             0.5/1.5 MB 5.8 MB/s eta 0:00:01
     -------------------------------          1.1/1.5 MB 9.1 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 9.3 MB/s eta 0:00:00
                                              0.0/139.3 kB ? eta -:--:--
     -------------------------------------- 139.3/139.3 kB 8.1 MB/s eta 0:00:00
                                              0.0/82.1 kB ? eta -:--:--
     ---------------------------------------- 82.1/82.1 kB ? eta 0:00:00
                                              0.0/147.8 kB ? eta -:--:--
     -------------------------------------- 147.8/147.8 kB 8.6 MB/s eta 0:00:00
                                              0.0/12.3 MB ? eta -:--:--
     ---                                      1.0/12.3 MB 31.4 MB/s eta 0:00:01
     -------- 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 2.1.1 requires sentencepiece, which is not installed.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.34.121 which is incompatible.


In [7]:
!pip install --upgrade boto3 botocore

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from botocore.config import Config

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\16479\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
# Set the AWS region
region = 'us-east-2'  # Replace with your desired region

In [3]:
# Initialize boto3 session and SageMaker session
session = boto3.Session(region_name=region)
s3_client = session.client('s3', config=Config(region_name=region))
sagemaker_session = sagemaker.Session(boto_session=session)

In [4]:
# Define S3 bucket and data locations
bucket_name = 'sagemaker-studio-211125530280-eyop3jjgekp'  # Replace with your actual bucket name
prefix = ''
train_data_local_path = 'C:/Users/16479/OneDrive/Desktop/train.csv'
test_data_local_path = 'C:/Users/16479/OneDrive/Desktop/test.csv'

In [5]:
# Create S3 bucket (if it doesn't already exist)
try:
    s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
    print(f'Bucket {bucket_name} created successfully.')
except s3_client.exceptions.BucketAlreadyOwnedByYou:
    print(f'Bucket {bucket_name} already exists.')

Bucket sagemaker-studio-211125530280-eyop3jjgekp already exists.


In [35]:
train_path = f's3://{bucket_name}/train.csv'
test_path = f's3://{bucket_name}/test.csv'

In [36]:
from sagemaker.estimator import Estimator

# Define the XGBoost container image
container = sagemaker.image_uris.retrieve('xgboost', sagemaker_session.boto_region_name, version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [37]:
sagemaker_session = sagemaker.Session()

# Define the IAM role
role = 'arn:aws:iam::211125530280:role/SageMakerExecutionRole'

In [42]:
xgboost = Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket_name}/output',
    sagemaker_session=sagemaker_session
)

In [43]:
# Set hyperparameters (adjust as needed)
xgboost.set_hyperparameters(
    objective='binary:logistic',
    num_round=100
)

In [44]:
# Prepare the data inputs for the training job
train_input = TrainingInput(train_path, content_type='csv')
test_input = TrainingInput(test_path, content_type='csv')

In [45]:
# Start the training job
xgboost.fit({'train': train_input, 'validation': test_input})


INFO:sagemaker:Creating training-job with name: xgboost-2024-06-07-17-59-21-747


2024-06-07 17:59:27 Starting - Starting the training job...
2024-06-07 17:59:41 Starting - Preparing the instances for training...
2024-06-07 18:00:09 Downloading - Downloading input data...
2024-06-07 18:00:49 Downloading - Downloading the training image...
2024-06-07 18:01:29 Training - Training image download completed. Training in progress....
2024-06-07 18:01:48 Uploading - Uploading generated training model
2024-06-07 18:01:48 Failed - Training job failed
..

UnexpectedStatusException: Error for Training job xgboost-2024-06-07-17-59-21-747: Failed. Reason: ClientError: Non-numeric value 's' found in the header line 'sepal length (cm),sepal width (cm),petal length (c...' of file 'train.csv'. CSV format require no header line in it. If header line is already removed, XGBoost does not accept non-numeric value in the data., exit code: 1